In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd 
import wandb
import sys

%config Completer.use_jedi = False
%matplotlib inline

sys.path.append("../../scripts/")
import style

In [ ]:
api = wandb.Api()
entity, project = "INPUT_YOUR_ENTITY", "curl"
runs = api.runs(entity + "/" + project) 

In [ ]:
datasets = ["wiki3029", "cifar10", "cifar100"]
classifier = "mean"

eval_records = []
contrastive_records = []
contrastive_eval_records = []

for run in runs: 
    
    if "hydra_path" not in run.config:
        continue

    if "k36049" not in run.config["hydra_path"]:
        continue
        
    dataset = run.config["dataset.name"]
    if dataset not in datasets:
        continue
        
    if run.config["name"] == classifier:
        
        if not run.config["normalize"]:
            continue

        extracted_keys = ("supervised_test_acc", "supervised_val_acc", "supervised_test_loss")
        
        for k in extracted_keys:
            run.config[k] = run.summary[k]        

        eval_records.append(run.config)
        
    elif run.config["name"] == "contrastive":

        v = run.config["hydra_path"] + "/" + run.config["output_model_name"]
        run.config["target_weight_file"] = v
        contrastive_records.append(run.config)

    elif run.config["name"] == "contrastive_eval":
        
        extracted_keys = ("contrastive_val_loss", "contrastive_test_loss")
        
        for k in extracted_keys:
            run.config[k] = run.summary[k]

        contrastive_eval_records.append(run.config)

In [ ]:
contrastive_df = pd.DataFrame.from_records(contrastive_records)
eval_df = pd.DataFrame.from_records(eval_records)
contrastive_eval_df = pd.DataFrame.from_records(contrastive_eval_records)


In [ ]:
merged_df = contrastive_df.merge(eval_df, how="inner", on="target_weight_file", suffixes=("", "_y"))

used_columns = [
    "seed", "dataset.name", "dataset.num_used_classes", "optimizer.lr", "loss.neg_size",
    "supervised_test_loss", "supervised_val_loss", "supervised_val_acc", "target_weight_file"
]
merged_df.drop(labels=[k for k in merged_df.keys() if k not in used_columns ], inplace=True, axis=1)
merged_df.head(1)

In [ ]:
merged_df = merged_df.merge(contrastive_eval_df, how="inner", on="target_weight_file", suffixes=("", "_y"))

used_columns = used_columns[:-1] + ["contrastive_test_loss", "supervised_test_loss"]
merged_df.drop(labels=[k for k in merged_df.keys() if k not in used_columns ], inplace=True, axis=1)
merged_df.head(1)

In [ ]:
removed_prefix_columns = []

# latex is unhappy with `_`, so replace them.
for c in merged_df.columns:
    removed_prefix_columns.append(c.split(".")[-1].replace("_", "-"))

merged_df.columns = removed_prefix_columns


In [ ]:
from plot_k_msuploss import ub_intercept
from compare_upper_bound import collision_term


def tau(k, c):
    """
    collision probability.
    """
    return (1. - (1. - 1. / c) ** k)

def ccp(k, c):    
    """
    coupon collector problem's probabilty;
    func2 on http://aquarius10.cse.kyutech.ac.jp/~otabe/shokugan/sg2.html
    """

    
    is_returned_single_value = isinstance(k, int)
    if is_returned_single_value:
        k = np.array([k])

    ret = []
    for _k in k:
        p = np.zeros(c + 1)
        p[0] = 1
        for j in range(_k):
            for i in range(c, -1, -1):
                p[i] = p[i] * i / c + p[i - 1] * (c - i + 1.) / c
                if i == 0:
                    p[0] = 0

        ret.append(p[c])

    ret = np.array(ret)

    if is_returned_single_value:
        return ret[0]
    else:
        return ret


def collision(k, c):
    """
    Collision term
    """
    return collision_term(np.array([k]), c)[0]

harmonic = lambda c: np.log(c) + 0.577


def arora(k, c, loss):
    if c > k:
        return None
    
    # since k is the number of negative samples,
    # we added the positive classes to k: k+1
    v = ccp(k+1, c)
    coeff = 1. / ((1. - tau(k, c)) * v)
    return coeff * (loss - collision(k, c))

def nozawa(k, c, loss):
    if c > k:
        return None  

    # since k is the number of negative samples,
    # we added the positive classes to k: k+1
    v = ccp(k+1, c)
    return (2. * loss - collision(k, c)) / v

def ash(k, c, loss):

    coeff = 2 * np.maximum(1, 2 * (c - 1) * harmonic(c - 1) / k) / (1 - tau(k, c))
    return coeff * (loss - collision(k, c))

def ours(k, c, loss):
    return loss + ub_intercept(k, c, b=1.0)


In [ ]:
rename = {"wiki3029": "Wiki-3029", "cifar100": "CIFAR-100", "cifar10": "CIFAR-10"}

for dataset in set(merged_df.name.values):

    idx = merged_df[merged_df["name"] == dataset].groupby(["seed", "num-used-classes", "neg-size"])["supervised-val-acc"].idxmax()
    df = merged_df.loc[idx,]
    
    negs = df["neg-size"].values
    seeds = df["seed"].values
    Cs = df["num-used-classes"].values
    test_contrastive_losses = df["contrastive-test-loss"].values
    test_sup_loss = df["supervised-test-loss"].values

    data = []
    for row in df.iterrows():
        row = row[1]
        seed = row["seed"]        
        K = row["neg-size"]
        C = row["num-used-classes"]
        supervised_loss = row["supervised-test-loss"]
        contrastive_loss = row["contrastive-test-loss"]
        new_d_row = [
            seed, C, K, supervised_loss,
            arora(K, C, contrastive_loss),
            nozawa(K, C, contrastive_loss),
            ash(K, C, contrastive_loss),
            ours(K, C, contrastive_loss)
        ]
        
        data.append(new_d_row)
    
    df = pd.DataFrame(data, columns=["seed", "C", "K", "sup_loss", "arora", "nozawa", "ash", "ours"])
    df = df.groupby(["C", "K"]).mean().reset_index()

    for c in set(df["C"].values):
        _df = df[df["C"] == c]
        marker = "o"
        plt.figure(figsize=(4, 3.5))
        plt.plot(_df["ours"].values, "-", color="C0", marker=marker, label="Ours")
        plt.plot(_df["arora"].values, "-", color="C1", marker=marker, label="Arora et al.")
        plt.plot(_df["nozawa"].values, "-", color="C2", marker=marker, label="Nozawa \& Sato")
        plt.plot(_df["ash"].values, "-", color="C3", marker=marker, label="Ash et al.")
        plt.plot(_df["sup_loss"].values, "x", color="black", markersize=13, label="Supervised loss")        
        plt.yscale("log")
        plt.xticks(np.arange(len(_df["K"].values)), _df["K"].values)
        plt.legend(fontsize=10)

        plt.xlabel(r"$K$ (Negative sample size)")
        plt.ylabel(r"Upper bound \& Supervised loss values")

        if dataset == "cifar10":
            plt.savefig("../../papers/figures/{}-upper-bound.pdf".format(dataset), bbox_inches="tight")

        # show title only for notebook
        title = "{}, C={}".format(rename[dataset], c)
        plt.title(title)
        plt.show()


# Closer look of `ours` and `supervised loss`

In [ ]:
rename = {"wiki3029": "Wiki-3029", "cifar100": "CIFAR-100", "cifar10": "CIFAR-10"}

for dataset in set(merged_df.name.values):

    idx = merged_df[merged_df["name"] == dataset].groupby(["seed", "num-used-classes", "neg-size"])["supervised-val-acc"].idxmax()
    df = merged_df.loc[idx,]
    
    negs = df["neg-size"].values
    seeds = df["seed"].values
    Cs = df["num-used-classes"].values
    test_contrastive_losses = df["contrastive-test-loss"].values
    test_sup_loss = df["supervised-test-loss"].values

    data = []
    for row in df.iterrows():
        row = row[1]
        seed = row["seed"]        
        K = row["neg-size"]
        C = row["num-used-classes"]
        supervised_loss = row["supervised-test-loss"]
        contrastive_loss = row["contrastive-test-loss"]
        new_d_row = [
            seed, C, K, supervised_loss,
            ours(K, C, contrastive_loss)
        ]
        
        data.append(new_d_row)
    
    df = pd.DataFrame(data, columns=["seed", "C", "K", "sup_loss", "ours"])
    mean = df.groupby(["C", "K"]).mean().reset_index()
    std = df.groupby(["C", "K"]).std().reset_index()    

    for c in set(df["C"].values):
        _df = mean[mean["C"] == c]
        _df_std = std[std["C"] == c]        
        marker = "o"
        plt.figure(figsize=(4, 3.5))
        plt.errorbar(np.arange(len(_df["ours"].values)), _df["ours"].values,  yerr=_df_std["ours"].values, color="C0", marker=marker, capsize=5, label="Ours")
        plt.errorbar(np.arange(len(_df["sup_loss"].values)), _df["sup_loss"].values, ls="", yerr=_df_std["sup_loss"].values, color="black", marker="x", markersize=8, capsize=5, label="Supervised loss")
        plt.xticks(np.arange(len(_df["K"].values)), _df["K"].values)
        plt.legend(fontsize=10)
        plt.xlabel(r"$K$")

        if dataset != "wiki3029":
            plt.savefig("../../papers/figures/closer-look-{}-upper-bound.pdf".format(dataset))

        # show title only for notebook
        title = "{}, $C={}$".format(rename[dataset], c)
        plt.title(title)
        plt.show()
